In [1]:
!unzip competitive-data-science-predict-future-sales.zip

Archive:  competitive-data-science-predict-future-sales.zip
  inflating: test.csv.gz             
  inflating: sample_submission.csv.gz  
  inflating: sales_train.csv.gz      
  inflating: shops.csv               
  inflating: item_categories.csv     
  inflating: items.csv               


In [0]:
import numpy as np
import pandas as pd 

In [0]:
'''
loading data
'''
sales_data = pd.read_csv('sales_train.csv.gz')
item_cat = pd.read_csv('item_categories.csv')
items = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
sample_submission = pd.read_csv('sample_submission.csv.gz')
test_data = pd.read_csv('test.csv.gz')

In [0]:
'''
creaning
'''
# object -> datetime
sales_data['date'] = pd.to_datetime(sales_data['date'], format='%d.%m.%Y')

In [0]:
# df of mean count per month
dataset = sales_data.pivot_table(
    index = ['shop_id', 'item_id'],
    columns = ['date_block_num'],
    values = ['item_cnt_day'],
    fill_value = 0
)
dataset.reset_index(inplace = True)

In [36]:
# extraciton for test_data
dataset = pd.merge(test_data,dataset,on = ['item_id','shop_id'],how = 'left')
dataset.fillna(0,inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [0]:
# drop unnecessary data
dataset.drop(['ID', 'shop_id', 'item_id'], inplace=True, axis=1)

In [39]:
dataset.head()

,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)","(item_cnt_day, 11)","(item_cnt_day, 12)","(item_cnt_day, 13)","(item_cnt_day, 14)","(item_cnt_day, 15)","(item_cnt_day, 16)","(item_cnt_day, 17)","(item_cnt_day, 18)","(item_cnt_day, 19)","(item_cnt_day, 20)","(item_cnt_day, 21)","(item_cnt_day, 22)","(item_cnt_day, 23)","(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,1.0,0.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# X_train is need to excepted last one (for num_0 ~ num_32 is uesed to predict)
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
# Y_train is last one (it is num_33)
Y_train = dataset.values[:,-1:]
# X_test is need to except first one (for num_1 ~ num_33 is uesed to predict)
X_test = np.expand_dims(dataset.values[:,1:], axis = 2)

In [67]:
'''
creating model
'''
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

Using TensorFlow backend.


In [68]:
model = Sequential()
model.add(LSTM(units = 64, input_shape = (33, 1)))
model.add(Dropout(0.4))
model.add(Dense(1))

model.compile(
    loss = 'mse',
    optimizer = 'adam',
    metrics = ['mean_squared_error']
)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [69]:
'''
learning
'''
model.fit(X_train,Y_train,batch_size = 4096,epochs = 10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
214200/214200 [==============================] - 7s 30us/step - loss: 1.3797 - mean_squared_error: 1.3797
Epoch 2/10
214200/214200 [==============================] - 3s 16us/step - loss: 1.3665 - mean_squared_error: 1.3665
Epoch 3/10
214200/214200 [==============================] - 4s 16us/step - loss: 1.3624 - mean_squared_error: 1.3624
Epoch 4/10
214200/214200 [==============================] - 3s 16us/step - loss: 1.3601 - mean_squared_error: 1.3601
Epoch 5/10
214200/214200 [==============================] - 3s 16us/step - loss: 1.3598 - mean_squared_error: 1.3598
Epoch 6/10
214200/214200 [==============================] - 4s 16us/step - loss: 1.3508 - mean_squared_error: 1.3508
Epoch 7/10
214200/214200 [==============================] - 3s 16us/step - loss: 1.3520 - mean_squared_error: 1.3520
Epoch 8/10
214200/214200 [==============================] - 3s 16us/step - loss: 1.3373 - mean_squared_error: 1.3373
Epoch 9/10
21420

In [0]:
'''
submission
'''
submission_pred = model.predict(X_test)

In [0]:
# every value between min0 ~ max20 (regulation)
submission_pred = submission_pred.clip(0, 20)
submission = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_pred.ravel()})
submission.to_csv('submission.csv',index = False)